In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import glob
import os
from math import log

In [11]:
allFiles = sorted(glob.glob("*.txt"),key=os.path.getmtime) # includes all files into a single stucture,
# Sorted by modification time so that the first measurement is first...
print(allFiles)

['PMT_Signal_280Deg_071717.txt', 'PMT_Background_280Deg_071717.txt', 'PMT_Background_275Deg_071717.txt', 'PMT_Signal_275Deg_071717.txt', 'PMT_Signal_270Deg_071717.txt', 'PMT_Background_270Deg_071717.txt', 'PMT_Background_265Deg_071717.txt', 'PMT_Signal_265Deg_071717.txt', 'PMT_Signal_260Deg_071717.txt', 'PMT_Background_260Deg_071717.txt', 'PMT2_Signal_280Deg_071717.txt', 'PMT2_Background_280Deg_071717.txt', 'PMT2_Background_275Deg_071717.txt', 'PMT2_Signal_275Deg_071717.txt', 'PMT2_Signal_270Deg_071717.txt', 'PMT2_Background_270Deg_071717.txt', 'PMT2_Background_265Deg_071717.txt', 'PMT2_Signal_265Deg_071717.txt', 'PMT2_Signal_260Deg_071717.txt', 'PMT2_Background_260Deg_071717.txt', 'PMT3_Signal_280Deg_071717.txt', 'PMT3_Background_280Deg_071717.txt', 'PMT3_Background_275Deg_071717.txt', 'PMT3_Signal_275Deg_071717.txt', 'PMT3_Signal_270Deg_071717.txt', 'PMT3_Background_270Deg_071717.txt', 'PMT3_Background_265Deg_071717.txt', 'PMT3_Signal_265Deg_071717.txt', 'PMT3_Signal_260Deg_071717.tx

In [3]:
frame = pd.DataFrame()
df = pd.DataFrame()

In [4]:
for i,file_ in enumerate(allFiles):
#    f = open(file_,'r')
#     for j,line in enumerate(f):
#         if j == 1:
#             date = line[14:22]
#             break
    df = pd.read_csv(file_,index_col = None, header=None, skiprows = 10, skipfooter = 1, delimiter = '\t')
#     frame[file_] = df[5]
    df['file']=file_
    df['timeDelay']= df[1]
    df['PDvalue'] = df[3]
#     df['date']=date
    if i < 1:
        frame = df
    else:
        frame = frame.append(df)

/Users/sanjaykhatri/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.


In [5]:
times = pd.Series()
PDvalues = pd.Series()
AvgPD50 = pd.Series()
itr = 0.000125
offset = 0.023
for i in range(0,100):
    time = frame[1][i]
    PDvalue = frame[3][i]
    times = times.append(time)
    PDvalues = PDvalues.append(PDvalue)
#print(len(PDvalues))
PDvalues = PDvalues.values.reshape(100,len(allFiles))
Background = np.array([PDvalues[:,1],PDvalues[:,2],PDvalues[:,5],PDvalues[:,6],PDvalues[:,9],\
                       PDvalues[:,21],PDvalues[:,22],PDvalues[:,25],PDvalues[:,26],PDvalues[:,29],\
                       PDvalues[:,31],PDvalues[:,32],PDvalues[:,35],PDvalues[:,36],PDvalues[:,39],\
                       PDvalues[:,41],PDvalues[:,42],PDvalues[:,45],PDvalues[:,46],PDvalues[:,49]])
Background_mean = np.mean(Background, axis = 1)
PMT_Raw = np.array([PDvalues[:,0],PDvalues[:,3],PDvalues[:,4],PDvalues[:,7],PDvalues[:,8],\
                    PDvalues[:,20],PDvalues[:,23],PDvalues[:,24],PDvalues[:,27],PDvalues[:,28],\
                    PDvalues[:,30],PDvalues[:,33],PDvalues[:,34],PDvalues[:,37],PDvalues[:,38],\
                    PDvalues[:,40],PDvalues[:,43],PDvalues[:,44],PDvalues[:,47],PDvalues[:,48]])
MeanPMT1 = np.mean(PMT_Raw,axis = 1)

PMT_Net = MeanPMT1 - Background_mean
MeanNET = np.reshape(PMT_Net,(4,5)) # As you add more files, change the 4 to the number of runs. 7 is b/c I took 7 different angles
MeanNET2 = np.mean(MeanNET,axis =0)
THzEnergy = np.array([11,10,8,6,4.5])
PMT_STD = np.std(MeanNET,axis = 0)
PMT_Err = PMT_STD/(4**0.5)
w = (PMT_Err/(MeanNET2*log(10)))**-2

print(Background_mean)
print(MeanPMT1)
print(MeanNET)
#print(PMT_)
#print(MeanNET2)
LogPMT = np.log10(np.absolute(MeanNET2))
LogTHz = np.log10(np.absolute(THzEnergy))
print(LogPMT)

[-0.224645 -0.210712 -0.194199 -0.1816   -0.180882 -0.200697 -0.198974
 -0.189569 -0.181136 -0.170216 -0.201327 -0.196883 -0.183178 -0.171792
 -0.174529 -0.198239 -0.191966 -0.182369 -0.170356 -0.161154]
[-0.259974 -0.249862 -0.231133 -0.209298 -0.19886  -0.243399 -0.236447
 -0.214466 -0.196183 -0.180476 -0.241802 -0.23357  -0.209388 -0.192573
 -0.187463 -0.233269 -0.226209 -0.202422 -0.191129 -0.183088]
[[-0.035329 -0.03915  -0.036934 -0.027698 -0.017978]
 [-0.042702 -0.037473 -0.024897 -0.015047 -0.01026 ]
 [-0.040475 -0.036687 -0.02621  -0.020781 -0.012934]
 [-0.03503  -0.034243 -0.020053 -0.020773 -0.021934]]
[-1.41584977 -1.43311195 -1.5682584  -1.67623757 -1.80198934]


In [6]:
Delta = sum(w)*sum(w*LogTHz**2) - (sum(w*LogTHz))**2
A = (sum(w*LogTHz**2)*sum(w*LogPMT) - sum(w*LogTHz)*sum(w*LogTHz*LogPMT))/Delta
B = (sum(w)*sum(w*LogTHz*LogPMT) - (sum(w*LogPMT)*sum(w*LogTHz)))/Delta
# Least Squares Fitting Parameters above
print(w)
print(A)
print(B)
err = (((len(LogPMT)-2)**-1) * sum((LogPMT - A -B*LogTHz)**2)**0.5)
errorA = (sum(w*LogTHz**2)/Delta)**0.5
errorB = (sum(w)/Delta)**0.5
print(errorA)
print(errorB)

[ 2866.60505322  9238.06385888   407.58716158   468.74434435   259.79241201]
-2.47229420705
1.031871012
0.126141684888
0.126943798409


In [9]:
plt.plot(LogTHz, A + B*LogTHz,'r',label = 'Fit: y=1.03x -2.47')
plt.plot(LogTHz, LogPMT,'b.', label = 'Data')
plt.grid(True)
plt.xlabel('Log(THz Energy)')
plt.ylabel('Log(PMT Signal)')
plt.legend(loc = 'upper left')
plt.show()
#plt.savefig('PMT_SignalVsTHzEnergy_07172017.png')